In [1]:
# ============================================
# Cell 0: 実験件数の設定（phish/random=100, benign=100, benign_hard=100）
# ============================================
# [ChangeLog] 2025-12-15: N_* に ALL/0/-1 を指定すると全件処理できるように拡張

import os

# ------------------------------------------------------------
# ▼▼▼ ここだけ編集すればOK（優先度: 環境変数 > ここ） ▼▼▼
#   - "500" のように数値 → その件数だけ処理
#   - "ALL" / 0 / -1     → 全件処理（制限なし）
# ------------------------------------------------------------
DEFAULT_N_SAMPLE = "500"
DEFAULT_N_BENIGN_SAMPLE = DEFAULT_N_SAMPLE
DEFAULT_N_BENIGN_HARD_SAMPLE = "500"

def _normalize_n_env(key: str, default: str) -> str:
    """環境変数 N_* を正規化して os.environ に入れる。
    - 1以上: その件数を処理
    - 0 / -1 / 'ALL' / '*' / 'FULL': 全件処理（制限なし）
    """
    raw = os.getenv(key, default)
    s = str(raw).strip()

    # 人間向けの指定（ALL 等）→ 数値センチネルへ
    if s.upper() in ("ALL", "*", "FULL"):
        return "-1"

    # 数値化（変な値なら default にフォールバック）
    try:
        n = int(float(s))
    except Exception:
        try:
            n = int(default)
        except Exception:
            n = -1
    # ★ 0 / 負値 は ALL 扱い（-1 に寄せる）
    return "-1" if n <= 0 else str(n)
def _n_label(n_str: str) -> str:
    """表示用: -1/0 以下は ALL と見せる"""
    try:
        n = int(str(n_str).strip())
    except Exception:
        return str(n_str)
    return "ALL" if n <= 0 else str(n)

# 3群の件数
os.environ["N_SAMPLE"] = _normalize_n_env("N_SAMPLE", DEFAULT_N_SAMPLE)
os.environ["N_BENIGN_SAMPLE"] = _normalize_n_env("N_BENIGN_SAMPLE", DEFAULT_N_BENIGN_SAMPLE)
os.environ["N_BENIGN_HARD_SAMPLE"] = _normalize_n_env("N_BENIGN_HARD_SAMPLE", DEFAULT_N_BENIGN_HARD_SAMPLE)

print("[SET] N_SAMPLE=", _n_label(os.environ["N_SAMPLE"]),
      "N_BENIGN_SAMPLE=", _n_label(os.environ["N_BENIGN_SAMPLE"]),
      "N_BENIGN_HARD_SAMPLE=", _n_label(os.environ["N_BENIGN_HARD_SAMPLE"]),)


[SET] N_SAMPLE= 500 N_BENIGN_SAMPLE= 500 N_BENIGN_HARD_SAMPLE= 500


In [2]:
# ============================================
# Cell 1: 環境設定と初期化（Cell0方式でRUN_ID決定）
# ============================================

import os
import sys
import json
import pickle
import pandas as pd
import numpy as np
import time
import importlib
from pathlib import Path
from typing import Dict, Any, List, Optional
import datetime
import run_id_registry as runreg
import importlib
import _compat.paths as paths
# ---------------------------------------------------------
# ▼▼▼ Evaluation sample size (variable) ▼▼▼
# ---------------------------------------------------------
N_SAMPLE = int(os.getenv("N_SAMPLE", "300"))
N_BENIGN_SAMPLE = int(os.getenv("N_BENIGN_SAMPLE", str(N_SAMPLE)))
RANDOM_STATE = int(os.getenv("RANDOM_STATE", "42"))
def _n_label(n: int) -> str:
    return "ALL" if int(n) <= 0 else str(int(n))
print(f"[INFO] N_SAMPLE={_n_label(N_SAMPLE)}, N_BENIGN_SAMPLE={_n_label(N_BENIGN_SAMPLE)}, RANDOM_STATE={RANDOM_STATE}")

# ---------------------------------------------------------
# プロジェクトルート推定
# ---------------------------------------------------------
BASE_DIR = Path(os.environ.get("NEXTSTEP_BASE_DIR", ".")).resolve()
sys.path.insert(0, str(BASE_DIR))

# [ChangeLog] 2025-12-16: sys.path から BASE_DIR/phishing_agent を除去（重複 import を防ぎ、Phase6 wiring を確実化）
# NOTE: パッケージ import を安定させるため、sys.path には「プロジェクトルート(BASE_DIR)」だけを入れる。
#       BASE_DIR/phishing_agent を入れると `langgraph_module` などがトップレベル import され、
#       `phishing_agent.langgraph_module` と二重ロードになり Phase6 のモンキーパッチが効かないことがある。
phishing_agent_path = BASE_DIR / "phishing_agent"
if str(BASE_DIR) not in sys.path:
    sys.path.insert(0, str(BASE_DIR))
# 念のため誤ったパスを消す（存在していれば）
if str(phishing_agent_path) in sys.path:
    sys.path.remove(str(phishing_agent_path))

# ---------------------------------------------------------
# ▼▼▼ RUN_ID の決定: 02以降共通 Cell0 と同じ方式 ▼▼▼
#   rid = runreg.bootstrap()  # env→run_id.txt→Part3→latest→新規
#   paths を reload して paths.RUN_ID を確定
#   rid と paths.RUN_ID がズレたら即検知
# ---------------------------------------------------------
import run_id_registry as runreg
rid = runreg.bootstrap()  # ここで env RUN_ID をセットする想定（念のため下で明示上書き）

# 念のため env を rid で固定（bootstrapがセットしていても害はない）
os.environ["RUN_ID"] = rid

# _compat.paths (または paths) を読み込み、env(RUN_ID) を反映させる
try:
    import _compat.paths as paths
except ImportError:
    import paths as paths  # fallback

importlib.reload(paths)  # Cell0方式（必要なら2回でも可だが通常1回で十分）

# デバッグ表示＆ズレ検知
print("[NX] RUN_ID =", rid, "| paths.RUN_ID =", paths.RUN_ID)
assert paths.RUN_ID == rid, f"RUN_ID mismatch: rid={rid} paths.RUN_ID={paths.RUN_ID}"

# 最終採用（ディレクトリ決定に使うのは paths.RUN_ID）
RUN_ID = paths.RUN_ID

# ---------------------------------------------------------
# artifacts/<RUN_ID>/... 配下の各ディレクトリ設定
# ---------------------------------------------------------
ARTIFACTS_DIR = Path(paths.ARTIFACTS) if hasattr(paths, "ARTIFACTS") else (BASE_DIR / "artifacts" / RUN_ID)
RAW_DIR       = Path(paths.compat_base_dirs["raw"])
PROCESSED_DIR = Path(paths.compat_base_dirs["data"])
MODELS_DIR    = Path(paths.compat_base_dirs["models"])
RESULTS_DIR   = Path(paths.compat_base_dirs["results"])
HANDOFF_DIR   = Path(paths.compat_base_dirs["handoff"])
LOGS_DIR      = Path(paths.compat_base_dirs["logs"])
TRACES_DIR    = Path(paths.compat_base_dirs["traces"])

print(f"[INFO] BASE_DIR      = {BASE_DIR}")
print(f"[INFO] RUN_ID        = {RUN_ID}")
print(f"[INFO] ARTIFACTS_DIR = {ARTIFACTS_DIR}")
print(f"[INFO] HANDOFF_DIR   = {HANDOFF_DIR}")

# Handoffディレクトリの存在確認
if HANDOFF_DIR.exists():
    print(f"[OK] Handoff directory found: {HANDOFF_DIR}")
    target_pkl = HANDOFF_DIR / "04-3_llm_tools_setup_with_tools.pkl"
    if target_pkl.exists():
        print(f"[OK] Target pickle found: {target_pkl.name}")
    else:
        print(f"[WARN] Target pickle NOT found in this directory.")
else:
    print(f"[WARN] Handoff directory NOT found. Check RUN_ID logic.")

# 出力用のタイムスタンプ
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
print(f"[INFO] Current Timestamp: {timestamp}")


[INFO] N_SAMPLE=500, N_BENIGN_SAMPLE=500, RANDOM_STATE=42
[NX] RUN_ID = 2026-01-10_140940 | paths.RUN_ID = 2026-01-10_140940
[INFO] BASE_DIR      = /data/hdd/asomura/nextstep
[INFO] RUN_ID        = 2026-01-10_140940
[INFO] ARTIFACTS_DIR = artifacts/2026-01-10_140940
[INFO] HANDOFF_DIR   = artifacts/2026-01-10_140940/handoff
[OK] Handoff directory found: artifacts/2026-01-10_140940/handoff
[WARN] Target pickle NOT found in this directory.
[INFO] Current Timestamp: 2026-01-10_190841


In [3]:
# ============================================
# Cell 2: External Dataの読み込みとBrand Keywords補強
# ============================================

import pickle

# handoffからexternal_dataを読み込み
handoff_path = HANDOFF_DIR / "04-3_llm_tools_setup_with_tools.pkl"

if not handoff_path.exists():
    raise FileNotFoundError(
        f"Handoff file not found: {handoff_path}\n"
        "04-3_llm_tools_setup_with_tools を先に実行して、artifacts/<RUN_ID>/handoff 配下に配置してください。"
    )

with open(handoff_path, 'rb') as f:
    external_data = pickle.load(f)

print(f"[INFO] external_data loaded from: {handoff_path}")

# --------------------------------------------------
# Brand Keywords 補強ロジック（元ノートの動作を踏襲）
# --------------------------------------------------

# 既存の brand_keywords を取得（無ければ空リスト）
brand_keywords = external_data.get("brand_keywords") or []
brands_lower = [b.lower() for b in brand_keywords]

# 論文・実験で重視したいブランド（不足していたものを補う）
essential_brands = [
    "mufg", "mufg-card", "mitsubishi-ufj", "三菱UFJ", "三菱ＵＦＪ",
    "smbc", "smbc-card", "三井住友カード", "三井住友銀行",
    "rakuten", "rakuten-card", "楽天カード", "楽天銀行",
    "amazon", "amazon-jp", "amazon.co.jp",
    "mercari", "メルカリ",
    "metamask", "binance", "bybit",
    # ▼▼▼ 追加分 ▼▼▼
    "sbi", "sbisec", "sumishin", "住信SBI",
    "telegram", "tg",
    "makuake",
    "aeon", "aeonbank", "イオン銀行",
    "jcb",
    "imad"
]

added_brands = []
for brand in essential_brands:
    if brand.lower() not in brands_lower:
        external_data.setdefault('brand_keywords', []).append(brand)
        added_brands.append(brand)
        brands_lower.append(brand.lower())

print(f"[INFO] Added {len(added_brands)} new brands")
print(f"[INFO] Total brand_keywords: {len(external_data['brand_keywords'])}")

# 主要ブランドの確認表示
check_brands = ["mufg", "smbc", "amazon", "mercari", "rakuten", "metamask"]
print("\n[Brand Check]")
for brand in check_brands:
    exists = brand in [b.lower() for b in external_data['brand_keywords']]
    status = "✅" if exists else "❌"
    print(f"  {brand:15} {status}")

FileNotFoundError: Handoff file not found: artifacts/2026-01-10_140940/handoff/04-3_llm_tools_setup_with_tools.pkl
04-3_llm_tools_setup_with_tools を先に実行して、artifacts/<RUN_ID>/handoff 配下に配置してください。

In [ ]:
# ============================================
# Cell 2の最後に追加: 04-2から不足データを補完
# ============================================

print("\n" + "=" * 80)
print("[INFO] Loading additional data from 04-2")
print("=" * 80)

# 04-2のパスを探索
pickle_04_2_paths = [
    HANDOFF_DIR / "04-2_statistical_analysis.pkl",
]

data_04_2 = None
for path in pickle_04_2_paths:
    if path.exists():
        print(f"[INFO] Found 04-2 at: {path}")
        try:
            with open(path, 'rb') as f:
                data_04_2 = pickle.load(f)
            print(f"[INFO] 04-2 loaded successfully")
            break
        except Exception as e:
            print(f"[WARN] Failed to load {path}: {e}")

if data_04_2 is None:
    print("[WARN] 04-2 not found, continuing with available data")
else:
    print(f"[INFO] 04-2 keys: {list(data_04_2.keys())[:10]}...")
    
    # TLD関連データを追加
    tld_keys = [
        'dangerous_tlds', 'DANGEROUS_TLDS',
        'legitimate_tlds', 'LEGITIMATE_TLDS',
        'neutral_tlds', 'NEUTRAL_TLDS',
        'phishing_tld_stats', 'TLD_STATS', 'tld_stats'
    ]
    
    added_count = 0
    for key in tld_keys:
        if key in data_04_2:
            # 標準化されたキー名にマッピング
            standard_key = key.lower()
            if 'dangerous' in standard_key:
                target_key = 'dangerous_tlds'
            elif 'legitimate' in standard_key:
                target_key = 'legitimate_tlds'
            elif 'neutral' in standard_key:
                target_key = 'neutral_tlds'
            elif 'tld' in standard_key and 'stat' in standard_key:
                target_key = 'phishing_tld_stats'
            else:
                target_key = key
            
            if target_key not in external_data:
                external_data[target_key] = data_04_2[key]
                value = data_04_2[key]
                size = len(value) if hasattr(value, '__len__') else 'N/A'
                print(f"  ✅ Added {target_key}: {type(value).__name__} (len={size})")
                added_count += 1
    
    # High Risk Words を追加
    hrw_keys = ['HIGH_RISK_WORDS', 'high_risk_words', 'high_risk']
    for key in hrw_keys:
        if key in data_04_2 and 'high_risk_words' not in external_data:
            external_data['high_risk_words'] = data_04_2[key]
            words = data_04_2[key]
            print(f"  ✅ Added high_risk_words: {type(words).__name__} (len={len(words)})")
            added_count += 1
            break
    
    # Known Domains を追加
    kd_keys = ['KNOWN_DOMAINS', 'known_domains', 'legitimate_domains']
    for key in kd_keys:
        if key in data_04_2 and 'known_domains' not in external_data:
            external_data['known_domains'] = data_04_2[key]
            domains = data_04_2[key]
            size = len(domains) if hasattr(domains, '__len__') else 'N/A'
            print(f"  ✅ Added known_domains: {type(domains).__name__} (len={size})")
            added_count += 1
            break
    
    print(f"\n[INFO] Added {added_count} data items from 04-2")

# 最終確認
print("\n" + "=" * 80)
print("[INFO] Final Data Availability Check")
print("=" * 80)

required_keys = {
    'brand_keywords': 'Brand keywords list',
    'cert_full_info_map': 'Certificate information',
    'dangerous_tlds': 'Dangerous TLDs',
    'legitimate_tlds': 'Legitimate TLDs',
    'neutral_tlds': 'Neutral TLDs',
    'phishing_tld_stats': 'TLD phishing statistics',
    'high_risk_words': 'High risk words',
    'known_domains': 'Known legitimate domains'
}

for key, description in required_keys.items():
    if key in external_data:
        value = external_data[key]
        size = len(value) if hasattr(value, '__len__') else 'N/A'
        print(f"  ✅ {description:30} (len={size})")
    else:
        print(f"  ❌ {description:30} NOT AVAILABLE")

In [ ]:
# ============================================
# Cell X: External Data の取得元と内容サマリ（健全性チェック）
# ============================================

# このノートブックで使う external_data は、原則ここまでで以下から構築されます：
#   1) artifacts/<RUN_ID>/handoff/04-3_llm_tools_setup_with_tools.pkl
#   2) (不足分がある場合) artifacts/<RUN_ID>/handoff/04-2_statistical_analysis.pkl
# その後、(必要に応じて) brand_keywords を補強しています。

from pprint import pprint

print("\n" + "="*80)
print("[External Data Summary]")
print("="*80)

required = [
    'brand_keywords',
    'cert_full_info_map',
    'dangerous_tlds',
    'legitimate_tlds',
    'neutral_tlds',
    'phishing_tld_stats',
    'high_risk_words',
    'known_domains',
]

src_hint = {
    '04-3': str(HANDOFF_DIR / '04-3_llm_tools_setup_with_tools.pkl'),
    '04-2': str(HANDOFF_DIR / '04-2_statistical_analysis.pkl'),
}
print("[INFO] Expected sources:")
pprint(src_hint)

missing = [k for k in required if k not in (external_data or {})]
if missing:
    print("\n[WARN] Missing keys in external_data:")
    pprint(missing)
else:
    print("\n[OK] All required keys exist.")

def _len(x):
    try:
        return len(x)
    except Exception:
        return 'N/A'

print("\n[Counts]")
for k in required:
    v = (external_data or {}).get(k)
    print(f"  {k:18} : {type(v).__name__:<12} len={_len(v)}")

# None/型ゆらぎ対策: external_data の *tlds が None の場合がある
def _as_list(v):
    if v is None:
        return []
    if isinstance(v, str):
        return [v]
    if isinstance(v, (list, tuple, set)):
        return list(v)
    # pandas Series / numpy array 等
    try:
        return list(v)
    except Exception:
        return []

# TLD セットの交差チェック（.com が dangerous に混入すると FP/FN 分析が壊れやすい）
D = {str(x).lower() for x in _as_list((external_data or {}).get('dangerous_tlds'))}
L = {str(x).lower() for x in _as_list((external_data or {}).get('legitimate_tlds'))}
N = {str(x).lower() for x in _as_list((external_data or {}).get('neutral_tlds'))}

print("\n[TLD Sanity]")
if 'com' in D:
    print("  ❌ WARNING: 'com' is inside dangerous_tlds. This will over-fire dangerous_tld for normal domains.")
else:
    print("  ✅ OK: 'com' is NOT in dangerous_tlds.")

overlap_DL = sorted(D & L)
overlap_DN = sorted(D & N)
overlap_LN = sorted(L & N)
if overlap_DL or overlap_DN or overlap_LN:
    print("  ⚠ Overlaps found (should ideally be empty):")
    if overlap_DL: print("    - dangerous ∩ legitimate:", overlap_DL[:20], "..." if len(overlap_DL)>20 else "")
    if overlap_DN: print("    - dangerous ∩ neutral   :", overlap_DN[:20], "..." if len(overlap_DN)>20 else "")
    if overlap_LN: print("    - legitimate ∩ neutral  :", overlap_LN[:20], "..." if len(overlap_LN)>20 else "")
else:
    print("  ✅ OK: No overlaps between dangerous/legitimate/neutral TLD sets.")

print("\n[Known Domains Note]")
kd = (external_data or {}).get('known_domains', {})
sample_items = list(kd.items())[:5] if isinstance(kd, dict) else []
print(f"  known_domains type={type(kd).__name__}, sample={sample_items}")
print("  NOTE: If known_domains is a 'seen list' (not a strict whitelist), it should NOT be used for safety mitigation.")

In [ ]:
# ============================================
# Cell 3: Randomサンプルデータの準備（件数=N_SAMPLE）
# ============================================

from urllib.parse import urlparse

# データ探索用ヘルパー関数
def _search_eval_df(obj):
    '''dict/list 再帰で DataFrame を探す'''
    if obj is None:
        return None
    if isinstance(obj, pd.DataFrame):
        return obj
    if isinstance(obj, dict):
        # よくあるキー直参照
        for k in ("false_negatives_df", "fn_df", "eval_df", "random_eval_df"):
            v = obj.get(k)
            if isinstance(v, pd.DataFrame):
                return v
        for v in obj.values():
            hit = _search_eval_df(v)
            if hit is not None:
                return hit
    if isinstance(obj, (list, tuple)):
        for v in obj:
            hit = _search_eval_df(v)
            if hit is not None:
                return hit
    return None

def _normalize_eval_df(df):
    '''列名の揺れに対応して domain, ml_probability の2列に正規化'''
    print("[DEBUG] columns:", list(df.columns))
    
    # lower → 元名のマップ
    lower2orig = {c.lower(): c for c in df.columns}
    
    # 1) domain候補
    domain_candidates = ["domain", "fqdn", "domain_name", "hostname", "host", "requested_host"]
    domain_key = None
    for key in domain_candidates:
        if key in lower2orig:
            domain_key = lower2orig[key]
            break
    
    if domain_key is None:
        # 部分一致
        for c in df.columns:
            if any(kw in c.lower() for kw in ["domain", "fqdn", "host", "url"]):
                domain_key = c
                print(f"[DEBUG] domain fallback: {c}")
                break
    
    # 2) ml_probability候補
    ml_candidates = ["ml_probability", "ml_prob", "probability", "prediction_proba", 
                     "score", "pred_proba", "proba", "confidence"]
    mlp_key = None
    for key in ml_candidates:
        if key in lower2orig:
            mlp_key = lower2orig[key]
            break
    
    if mlp_key is None:
        # floatカラムから推測
        float_cols = [c for c in df.columns if pd.api.types.is_float_dtype(df[c])]
        for c in float_cols:
            if any(kw in c.lower() for kw in ["prob", "score", "pred"]):
                mlp_key = c
                print(f"[DEBUG] ml_probability fallback: {c}")
                break
    
    if domain_key is None or mlp_key is None:
        print("[ERROR] could not infer domain/ml_probability columns")
        return None
    
    print(f"[DEBUG] chosen columns: domain={domain_key}, ml_prob={mlp_key}")
    
    tmp = df[[domain_key, mlp_key]].copy()
    tmp.columns = ["domain", "ml_probability"]
    
    # url→domainの正規化
    if "url" in domain_key.lower():
        def _to_domain(x):
            if not isinstance(x, str):
                return ""
            if "://" in x:
                netloc = urlparse(x).netloc
            else:
                netloc = x
            netloc = netloc.split("@")[-1].split(":")[0]
            return netloc.lower()
        tmp["domain"] = tmp["domain"].map(_to_domain)
    
    # クリーンアップ
    tmp["domain"] = tmp["domain"].astype(str).str.strip().str.lower()
    tmp = tmp[tmp["domain"].str.len() > 0]
    tmp["ml_probability"] = pd.to_numeric(tmp["ml_probability"], errors="coerce")
    tmp = tmp[(tmp["ml_probability"] >= 0.0) & (tmp["ml_probability"] <= 1.0)]
    tmp = tmp.dropna(subset=["ml_probability"]).reset_index(drop=True)
    
    return tmp

# Randomサンプルデータを取得
def get_random_domains(sample_size=None):
    '''Randomサンプルのドメインリストを取得（件数=sample_size）'''
    if sample_size is None:
        sample_size = N_SAMPLE
    sample_size = int(sample_size)
    
    # 方法1: 既存のCSVから
    csv_paths = [
    LOGS_DIR / "random_eval_domains_latest.csv",
]
    
    for csv_path in csv_paths:
        if csv_path.exists():
            print(f"[INFO] Loading existing CSV from: {csv_path}")
            df = pd.read_csv(csv_path)


            # ALL 指定（0/-1）なら、既存CSVが部分サンプルの可能性があるので pickle から再生成する
            if sample_size <= 0:
                print(f"[INFO] sample_size={sample_size} (ALL) requested; ignore cache and regenerate from pickle...")
                break
            # 既存CSVが要求件数と一致するならそのまま使う。
            # もし件数が違う場合は、pickle から再生成する（古いキャッシュを掴まないため）。
            if (sample_size > 0) and (len(df) == sample_size):
                return df

            if (sample_size > 0) and (len(df) > sample_size):
                print(f"[INFO] Existing CSV has n={len(df)} but requested n={sample_size}; downsampling.")
                return df.sample(n=sample_size, random_state=RANDOM_STATE).reset_index(drop=True)

            print(f"[INFO] Existing CSV has n={len(df)} but requested n={sample_size}; regenerating from pickle...")
            break
    
    # 方法2: pickleから生成（元のノートブックのロジック）
    print("[INFO] CSV not found, generating from pickle...")
    
    pickle_paths = [
    HANDOFF_DIR / "04-2_statistical_analysis.pkl",
    HANDOFF_DIR / "04-3_llm_tools_setup_with_tools.pkl",
]
    
    eval_source_df = None
    for pickle_path in pickle_paths:
        if not pickle_path.exists():
            continue
            
        print(f"[INFO] Trying: {pickle_path}")
        try:
            with open(pickle_path, 'rb') as f:
                obj = pickle.load(f)
            
            # DataFrameを探す
            raw_df = _search_eval_df(obj)
            if raw_df is not None and len(raw_df) > 0:
                print(f"[INFO] Found DataFrame with {len(raw_df)} rows")
                eval_source_df = raw_df
                break
        except Exception as e:
            print(f"[WARN] Failed to load {pickle_path}: {e}")
    
    if eval_source_df is None:
        raise RuntimeError(
            "評価用DataFrameが見つかりません。\n"
            "04-2 または 04-3 のpickleファイルを確認してください。"
        )
    
    # 正規化
    norm_df = _normalize_eval_df(eval_source_df)
    if norm_df is None:
        raise RuntimeError("DataFrameの正規化に失敗しました")
    
    print(f"[INFO] Source rows (normalized): {len(norm_df)}")
    
    # ランダムサンプリング（元のノートブックは固定シードなし）
    # [ChangeLog] 2025-12-15: sample_size<=0(ALL) の場合は全件を返す
    if sample_size <= 0:
        sample_n = len(norm_df)
        # 全件処理のときはシャッフルせず、そのまま（再現性・速度優先）
        sample_df = norm_df.reset_index(drop=True)
    else:
        sample_n = min(sample_size, len(norm_df))
        sample_df = norm_df.sample(n=sample_n).reset_index(drop=True)
    
    # CSV保存
    out_csv = LOGS_DIR / "random_eval_domains_latest.csv"
    LOGS_DIR.mkdir(parents=True, exist_ok=True)
    sample_df.to_csv(out_csv, index=False, encoding="utf-8")
    print(f"[INFO] Random sample saved to: {out_csv} (n={sample_n})")
    
    return sample_df

# Randomサンプルデータを取得
target_df = get_random_domains(sample_size=N_SAMPLE)
print(f"[INFO] Target sample loaded: {len(target_df)} domains (N_SAMPLE={N_SAMPLE})")
print(target_df.head())

# ML確率の分布を確認
print(f"\n[ML Probability Distribution]")
print(f"  Min:    {target_df['ml_probability'].min():.3f}")
print(f"  Max:    {target_df['ml_probability'].max():.3f}")
print(f"  Mean:   {target_df['ml_probability'].mean():.3f}")
print(f"  Median: {target_df['ml_probability'].median():.3f}")

# 高リスクドメイン（ML確率 > 0.4）の数
high_risk = target_df[target_df['ml_probability'] > 0.4]
print(f"  High risk (>0.4): {len(high_risk)} domains")


In [ ]:
# ============================================
# Cell 4: LangGraphエージェントの初期化（サンプル用・LLM必須版, config読み取り専用）
# ============================================

import sys
import os
import json
from pathlib import Path

# 環境設定
os.chdir(BASE_DIR)
# [ChangeLog] 2025-12-16: sys.path を整理（Phase6 wiring の二重 import 問題を回避）
# NOTE: sys.path には BASE_DIR のみを入れて package import を統一する。
phishing_agent_path = BASE_DIR / "phishing_agent"
if str(BASE_DIR) not in sys.path:
    sys.path.insert(0, str(BASE_DIR))
if str(phishing_agent_path) in sys.path:
    sys.path.remove(str(phishing_agent_path))

print("=" * 80)
print("AGENT INITIALIZATION WITH LLM (MANDATORY, READ-ONLY CONFIG)")
print("=" * 80)

# 1. _compat/config.json から LLM 設定を読む（※書き換えない）
print("\n[1] Loading LLM Configuration from _compat/config.json")
print("-" * 40)

cfg_path = BASE_DIR / "_compat" / "config.json"
cfg_json = {}
llm_cfg = {}

if not cfg_path.exists():
    print(f"❌ config.json not found at {cfg_path}")
else:
    try:
        with open(cfg_path, "r", encoding="utf-8") as f:
            cfg_json = json.load(f)
        llm_cfg = (cfg_json.get("llm") or {})
        print(f"✅ Loaded config.json from {cfg_path}")
    except Exception as e:
        print(f"❌ Failed to load config.json: {e}")
        cfg_json = {}
        llm_cfg = {}

# LLM設定のサマリ表示
if not llm_cfg:
    print("❌ 'llm' section not found in config.json")
else:
    provider   = llm_cfg.get("provider")
    base_url   = (
        llm_cfg.get("base_url")
        or llm_cfg.get("vllm_base_url")
        or llm_cfg.get("ollama_base_url")
    )
    model      = (
        llm_cfg.get("model")
        or llm_cfg.get("vllm_model")
        or llm_cfg.get("ollama_model")
    )
    enabled    = bool(llm_cfg.get("enabled", False))
    temperature = llm_cfg.get("temperature", 0.1)

    print("LLM config overview:")
    print(f"  enabled   : {enabled}")
    print(f"  provider  : {provider}")
    print(f"  base_url  : {base_url}")
    print(f"  model     : {model}")
    print(f"  temperature: {temperature}")

    if not enabled:
        print("⚠ LLM is disabled in config.json (llm.enabled=False)")
    if not base_url:
        print("⚠ llm.base_url / vllm_base_url / ollama_base_url is not set")
    if not model:
        print("⚠ llm.model / vllm_model / ollama_model is not set")

# external_data にも cfg を入れておく（読み取り結果だけ反映）
if "external_data" not in globals():
    external_data = {}
external_data.setdefault("cfg", {})
external_data["cfg"]["llm"] = llm_cfg

# 2. OPENAI_API_KEY の設定（vLLMの場合はダミーでOK）
print("\n[2] Setting OPENAI_API_KEY (dummy for vLLM / Ollama)")
print("-" * 40)
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = llm_cfg.get("api_key") or "dummy-key-for-local-llm"
    print("✅ OPENAI_API_KEY set (dummy or from config)")
else:
    print("ℹ OPENAI_API_KEY already set in environment")

# 3. phishpkg/旧モジュールをクリア（あれば）
print("\n[3] Clearing legacy 'phishpkg' modules (if any)")
print("-" * 40)
removed = 0
for key in list(sys.modules.keys()):
    if key.startswith("phishpkg"):
        del sys.modules[key]
        removed += 1
print(f"✅ Removed {removed} phishpkg modules from sys.modules")

# 4. Phase6 配線（LLM必須モード／fake_llm=False）
print("\n[4] Wiring Phase6 (LLM required, using config.json)")
print("-" * 40)

try:
    from phishing_agent.phase6_wiring import wire_phase6

    # CONFIG_JSON を明示しておくと安全
    cfg_env_path = str(cfg_path)
    os.environ["CONFIG_JSON"] = cfg_env_path

    # prefer_compat=True: _compat/config.json を優先
    # fake_llm=False    : 実際の LLM を使う前提（失敗時は例外 or フォールバックはphase6_wiring側ルール）
    wire_phase6(prefer_compat=True, fake_llm=False)
    print("✅ Phase6 wired with real LLM (using config.json)")

    # [ChangeLog] 2025-12-16: どのモジュールが読み込まれているか明示（取り違え防止）
    import phishing_agent.phase6_wiring as _p6w
    import phishing_agent.langgraph_module as _l4
    import inspect as _inspect
    print(f"  phase6_wiring.__file__      = {_p6w.__file__}")
    print(f"  langgraph_module.__file__   = {_l4.__file__}")
    try:
        _fd = _l4.LangGraphPhishingAgent._final_decision_node
        print(f"  final_decision_node.module  = {_fd.__module__}")
        print(f"  final_decision_node.file    = {_inspect.getsourcefile(_fd)}")
    except Exception as _e:
        print(f"  (warn) could not introspect final_decision_node: {_e}")
except Exception as e:
    print(f"❌ Phase6 wiring failed: {e}")
    raise

# 5. LangGraph エージェントをインポート
print("\n[5] Importing LangGraphPhishingAgent")
print("-" * 40)

from phishing_agent.langgraph_module import LangGraphPhishingAgent

print("✅ LangGraphPhishingAgent imported")

# 6. エージェント初期化（use_llm_decision=True が超重要）
print("\n[6] Initializing Agent (LLM mandatory, config-driven)")
print("-" * 40)

agent = LangGraphPhishingAgent(
    strict_mode=True,              # Graph全体としては strict=False（Phase6側でSOはstrictにしてOK）
    use_llm_selection=True,
    use_llm_decision=True,          # ★ final_decision で必ず LLM 経路を試す
    config_path=str(cfg_path),
    external_data=external_data,
)
print("✅ Agent initialized")

# 7. 動作確認テスト（LLMパス／debug_llm_final の確認）
print("\n[7] Quick Verification Test (single domain)")
print("-" * 40)

import time

test_domain = "test-amazon.com"
start = time.time()

try:
    result = agent.evaluate(test_domain, 0.35)
    elapsed = time.time() - start

    print(f"Test domain         : {test_domain}")
    print(f"  Time              : {elapsed:.2f}s")
    print(f"  success           : {result.get('success')}")
    print(f"  ai_is_phishing    : {result.get('ai_is_phishing')}")
    print(f"  ai_confidence     : {result.get('ai_confidence')}")
    print(f"  ai_risk_level     : {result.get('ai_risk_level')}")

    gs = result.get("graph_state") or {}
    dbg = gs.get("debug_llm_final") or {}
    print(f"  debug_llm_final   : {dbg}")

    # [ChangeLog] 2025-12-16: Phase6の有効化を「確実に」検証（Phase4でも path=="llm" は起こり得る）
    p6v = gs.get("phase6_policy_version")
    dt  = gs.get("decision_trace") or []
    print(f"  phase6_policy_version: {p6v}")
    try:
        _dt_len = len(dt) if isinstance(dt, list) else None
    except Exception:
        _dt_len = None
    print(f"  decision_trace_len    : {_dt_len}")

    if not p6v:
        raise RuntimeError(
            "Phase6 is NOT active: graph_state.phase6_policy_version is missing. "
            "Check sys.path/imports and ensure phase6_wiring is applied before agent init."
        )

    path = dbg.get("path")
    if path == "llm" and dbg.get("success") is True:
        print("  ✅ Phase6 LLM path is active (path='llm')")
    elif path == "fallback":
        print("  ⚠ Phase6 LLM failed; fallback path used (path='fallback')")
    else:
        print("  ℹ Phase6 LLM path status is unclear (see debug_llm_final above)")

except Exception as e:
    elapsed = time.time() - start
    print(f"  ❌ Test failed after {elapsed:.2f}s: {e}")

print("\n" + "=" * 80)
print("AGENT READY FOR EVALUATION (LLM MANDATORY, CONFIG-DRIVEN)")
print("=" * 80)
print("✅ LLM: Loaded from _compat/config.json")
print("✅ Phase6: wired (LLM経路あり、fallbackはphase6_wiring側ロジックに依存)")
print("✅ Agent: Initialized")
print("\nProceed to Cell 5 for RandomSample evaluation")


In [ ]:
# ============================================
# Cell 4.6: TLDリストの動的生成（データベース分析版）
# ※ DB接続必須。失敗時はエラー終了します。
# ============================================
import psycopg2
from psycopg2.extras import RealDictCursor
from collections import Counter
from urllib.parse import urlparse
import random
import numpy as np

print("🔧 Generating TLD data from Database Analysis...")

# 03_ai_agent_analysis.ipynb からの DB設定
DB_CONFIG = {
    'dbname': 'rapids_data',
    'user': 'postgres',
    'password': 'asomura',
    'host': 'localhost',
    'port': '5432'
}

# TLD抽出関数 (03からの移植)
def extract_tld(domain):
    """ドメインからTLDを抽出"""
    if not domain:
        return None
    # URLの場合はドメイン部分を抽出
    if '://' in domain:
        try:
            parsed = urlparse(domain)
            domain = parsed.netloc
        except:
            pass
    
    # ポート番号を削除
    domain = str(domain).split(':')[0]
    
    parts = domain.split('.')
    if len(parts) >= 2:
        # .co.jp, .ac.jp などの複合TLD対応
        if len(parts) >= 3 and parts[-2] in ['co', 'ac', 'or', 'ne', 'go']:
            return f'.{parts[-2]}.{parts[-1]}'
        else:
            return f'.{parts[-1]}'
    return None

try:
    # データベース接続テスト
    print(f"  🔌 Connecting to database: {DB_CONFIG['dbname']} at {DB_CONFIG['host']}...")
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor(cursor_factory=RealDictCursor)
    print("  ✅ Database connected successfully")

    # 1. フィッシングサイトの取得 (各ソースから)
    print("  📊 Analyzing phishing domains...")
    phishing_queries = [
        "SELECT cert_domain as domain FROM phishtank_entries WHERE cert_status = 'SUCCESS' AND cert_data IS NOT NULL",
        "SELECT domain FROM jpcert_phishing_urls WHERE status = 'SUCCESS' AND cert_data IS NOT NULL",
        "SELECT domain FROM certificates WHERE status = 'SUCCESS' AND cert_data IS NOT NULL"
    ]
    
    phishing_domains = []
    for query in phishing_queries:
        cur.execute(query)
        results = cur.fetchall()
        for row in results:
            if row['domain']: phishing_domains.append(row['domain'])
            
    if not phishing_domains:
        raise ValueError("No phishing domains found in database. Analysis cannot proceed.")

    # 2. 正常サイトの取得
    print("  📊 Analyzing trusted domains...")
    cur.execute("SELECT domain FROM trusted_certificates WHERE status = 'SUCCESS' AND cert_data IS NOT NULL")
    results = cur.fetchall()
    trusted_domains = [row['domain'] for row in results if row['domain']]
    
    conn.close()

    # 3. データバランス調整 (03と同様のロジック)
    # 重複除去
    phishing_domains_unique = list(set(phishing_domains))
    trusted_domains_unique = list(set(trusted_domains))
    
    print(f"    - Phishing unique: {len(phishing_domains_unique)}")
    print(f"    - Trusted unique: {len(trusted_domains_unique)}")
    
    # 少ない方に合わせてサンプリング
    random.seed(42)
    min_unique = min(len(phishing_domains_unique), len(trusted_domains_unique))
    
    if min_unique == 0:
        raise ValueError("Insufficient data for analysis (one of the datasets is empty).")
        
    phishing_balanced = random.sample(phishing_domains_unique, min(len(phishing_domains_unique), min_unique))
    trusted_balanced = random.sample(trusted_domains_unique, min(len(trusted_domains_unique), min_unique))

    # 4. TLD集計
    phishing_tlds = Counter([extract_tld(d) for d in phishing_balanced if extract_tld(d)])
    trusted_tlds = Counter([extract_tld(d) for d in trusted_balanced if extract_tld(d)])

    # 5. 危険度分析と分類
    dangerous_tlds = []
    
    for tld, phish_count in phishing_tlds.items():
        trust_count = trusted_tlds.get(tld, 0)
        # サンプル数が少なすぎるものは除外
        if phish_count >= 10:
            ratio = phish_count / (trust_count + 1)
            phish_pct = phish_count / len(phishing_balanced) * 100
            
            # 判定ロジック:
            # 1. 正常サイトで皆無(0件) かつ フィッシングで10件以上
            # 2. 比率が10倍以上 かつ フィッシング全体の0.1%以上
            if (trust_count == 0) or (ratio >= 10 and phish_pct >= 0.1):
                dangerous_tlds.append(tld)
    
    # ドットを除去して格納 (例: '.xyz' -> 'xyz')
    external_data['dangerous_tlds'] = [t.lstrip('.') for t in dangerous_tlds]

    # 正当なTLD
    legitimate_tlds = []
    for tld, count in trusted_tlds.most_common():
        if count >= 1000: # 統計的信頼性
            phish_count = phishing_tlds.get(tld, 0)
            ratio = phish_count / count
            if ratio < 0.5: # 正常サイトでの使用が2倍以上
                legitimate_tlds.append(tld)
    external_data['legitimate_tlds'] = [t.lstrip('.') for t in legitimate_tlds]

    # 中立的なTLD (主要TLDのうち、上記に入らなかったもの)
    neutral_candidates = ['.com', '.org', '.net', '.info', '.biz']
    neutral_tlds = []
    for tld in neutral_candidates:
        if tld not in dangerous_tlds and tld not in legitimate_tlds:
            neutral_tlds.append(tld)
    external_data['neutral_tlds'] = [t.lstrip('.') for t in neutral_tlds]
    
    # 統計情報を保存 (生のCounterオブジェクト)
    external_data['phishing_tld_stats'] = phishing_tlds

    print(f"  ✅ Generated {len(external_data['dangerous_tlds'])} dangerous TLDs")
    print(f"  ✅ Generated {len(external_data['legitimate_tlds'])} legitimate TLDs")

    # エージェント内の参照更新
    if 'agent' in globals():
        agent.external_data = external_data
        print("  ✅ Agent external_data updated")
        
    print("✅ TLD data patched via Database Analysis. Ready for evaluation.")

except Exception as e:
    print(f"\n❌ DATABASE ANALYSIS FAILED: {e}")
    print("⛔ Stopping execution to prevent inaccurate results using fallback data.")
    # 明示的にエラーを発生させて処理を止める
    raise RuntimeError("Database connection or analysis failed. Please check DB connection and rerun this cell.") from e


In [ ]:
# ============================================
# Cell 5: サンプル評価実行（件数=len(target_df), 超詳細デバッグログ版）
# ============================================

import time
import pandas as pd
from datetime import datetime
import io, sys, json, os
from contextlib import redirect_stdout, redirect_stderr

print(f"[INFO] Starting FULL AGENT evaluation of {len(target_df)} domains (N_SAMPLE={N_SAMPLE})...")
print("=" * 80)

# LLM設定の確認
if 'agent' in globals() and hasattr(agent, 'llm_config'):
    llm_config = agent.llm_config
    if getattr(llm_config, "enabled", False):
        print(f"[INFO] LLM initialized: {llm_config.model}")
    else:
        print("[WARNING] LLM is disabled - results may be limited")
else:
    print("[WARNING] Agent not properly initialized")

# --- Code fingerprint (reproducibility / verification) ------------------------
# [ChangeLog] 2025-12-17: Add code fingerprint (module __file__ + sha256) to each output row.
import hashlib
import importlib
from pathlib import Path
import sys
import datetime as _dt

def _sha256_of_file(path: str, chunk_size: int = 1024 * 1024) -> str:
    """Return sha256 hex digest for a local file. Never raises."""
    try:
        h = hashlib.sha256()
        with open(path, "rb") as f:
            for chunk in iter(lambda: f.read(chunk_size), b""):
                h.update(chunk)
        return h.hexdigest()
    except Exception as e:
        return f"ERROR:{type(e).__name__}"

def _module_file_and_sha(modname: str):
    """Import module and return (file_path, sha256, module_obj)."""
    try:
        mod = importlib.import_module(modname)
        f = getattr(mod, "__file__", None)
        if f:
            f = str(Path(f).resolve())
            sha = _sha256_of_file(f)
        else:
            sha = None
        return f, sha, mod
    except Exception as e:
        return None, f"IMPORT_ERROR:{type(e).__name__}", None

def make_code_fingerprint() -> dict:
    fp = {}
    fp["eval_id"] = _dt.datetime.now().strftime("%Y-%m-%d_%H%M%S")

    # Core modules used by final decision path
    p6_file, p6_sha, _p6_mod = _module_file_and_sha("phishing_agent.phase6_wiring")
    ld_file, ld_sha, ld_mod = _module_file_and_sha("phishing_agent.llm_final_decision")
    lg_file, lg_sha, _lg_mod = _module_file_and_sha("phishing_agent.langgraph_module")

    fp["phase6_wiring_file"] = p6_file
    fp["phase6_wiring_sha256"] = p6_sha
    fp["llm_final_decision_file"] = ld_file
    fp["llm_final_decision_sha256"] = ld_sha
    fp["langgraph_module_file"] = lg_file
    fp["langgraph_module_sha256"] = lg_sha

    fp["phase6_policy_version_code"] = getattr(ld_mod, "PHASE6_POLICY_VERSION", None) if ld_mod else None
    fp["python"] = sys.version.split()[0]
    fp["dual_import_langgraph_module"] = ("langgraph_module" in sys.modules) and ("phishing_agent.langgraph_module" in sys.modules)

    return fp

# Compute once per notebook run (avoid per-domain overhead)
if "CODE_FINGERPRINT" not in globals():
    CODE_FINGERPRINT = make_code_fingerprint()
    CODE_FP_ROW = {
        "eval_id": CODE_FINGERPRINT.get("eval_id"),
        "phase6_policy_version_code": CODE_FINGERPRINT.get("phase6_policy_version_code"),
        "phase6_wiring_file": CODE_FINGERPRINT.get("phase6_wiring_file"),
        "phase6_wiring_sha256": CODE_FINGERPRINT.get("phase6_wiring_sha256"),
        "llm_final_decision_file": CODE_FINGERPRINT.get("llm_final_decision_file"),
        "llm_final_decision_sha256": CODE_FINGERPRINT.get("llm_final_decision_sha256"),
        "langgraph_module_file": CODE_FINGERPRINT.get("langgraph_module_file"),
        "langgraph_module_sha256": CODE_FINGERPRINT.get("langgraph_module_sha256"),
        "dual_import_langgraph_module": CODE_FINGERPRINT.get("dual_import_langgraph_module"),
    }
    print("[INFO] code_fingerprint:", CODE_FP_ROW)
else:
    # Reuse existing in case notebook cells are re-run
    CODE_FP_ROW = globals().get("CODE_FP_ROW", {})

results = []
start_time = time.time()
# --- 保存設定: 500件ごとに追記（分割ファイルなし） -----------------
SAVE_EVERY_ROWS = 500  # 要件: 500件ごとに保存（追記）
if "LOGS_DIR" not in globals() or not globals().get("LOGS_DIR"):
    raise RuntimeError("LOGS_DIR is not set. Run Cell 1 to initialize run_id/paths first.")

OUT_DIR = Path(globals()["LOGS_DIR"])
OUT_DIR.mkdir(parents=True, exist_ok=True)

# eval_id は Notebook実行単位で固定（CODE_FP_ROW があればそれを使う）
_eval_id = None
if "CODE_FP_ROW" in globals() and isinstance(globals().get("CODE_FP_ROW"), dict):
    _eval_id = globals()["CODE_FP_ROW"].get("eval_id")
if not _eval_id:
    _eval_id = datetime.now().strftime("%Y-%m-%d_%H%M%S")

_ts = datetime.now().strftime("%Y-%m-%d_%H%M%S")

# 追記先（run_id/logs 配下へ）
full_eval_path = OUT_DIR / f"random{len(target_df)}_full_eval__evalid_{_eval_id}__ts_{_ts}.csv"
print(f"[INFO] Results will be appended to: {full_eval_path} (flush every {SAVE_EVERY_ROWS} rows)")

_chunk_buf = []
_written_header = False
_written_rows = 0

def _flush_chunk_if_needed(force: bool = False):
    """_chunk_buf を full_eval_path に追記保存（ヘッダは最初だけ）。"""
    global _written_header, _written_rows
    if (not force) and (len(_chunk_buf) < SAVE_EVERY_ROWS):
        return
    if not _chunk_buf:
        return
    df_chunk = pd.DataFrame(_chunk_buf)
    df_chunk.to_csv(full_eval_path, mode="a", header=(not _written_header), index=False)
    _written_header = True
    _written_rows += int(len(_chunk_buf))
    _chunk_buf.clear()

# --- Notebook画面出力の制御（debug_log の情報量は維持） -----------------
# [ChangeLog] 2025-12-15: 大量件数でもJupyterの出力が重くならないよう、画面出力を間引く
TOTAL_N = len(target_df)

# デフォルト: 少数件は従来どおり詳細表示 / 多数件はコンパクト表示
SHOW_DOMAIN_LOG_ON_SCREEN = (TOTAL_N <= 20)

# 環境変数で上書き可: 1/true/yes/on で詳細表示
_env = os.getenv("SHOW_DOMAIN_LOG_ON_SCREEN")
if _env is not None:
    SHOW_DOMAIN_LOG_ON_SCREEN = str(_env).strip().lower() in ("1", "true", "yes", "y", "on")

# 進捗表示の更新頻度（画面を軽くするため、件数に応じて自動調整）
if TOTAL_N <= 30:
    PROGRESS_EVERY = 1
elif TOTAL_N <= 300:
    PROGRESS_EVERY = 10
elif TOTAL_N <= 3000:
    PROGRESS_EVERY = 25
else:
    PROGRESS_EVERY = 50

# Jupyterなら clear_output で表示を置き換え（ログはCSV側に残る）
_USE_CLEAR_OUTPUT = (not SHOW_DOMAIN_LOG_ON_SCREEN)
try:
    from IPython.display import clear_output  # type: ignore
    _HAS_CLEAR_OUTPUT = True
except Exception:
    _HAS_CLEAR_OUTPUT = False

print(f"[INFO] Console output mode: {'VERBOSE' if SHOW_DOMAIN_LOG_ON_SCREEN else 'COMPACT'} "
      f"(PROGRESS_EVERY={PROGRESS_EVERY}, clear_output={_USE_CLEAR_OUTPUT and _HAS_CLEAR_OUTPUT})")

# 進捗カウンタ（画面表示用）
_screen_counts = {"phish": 0, "benign": 0, "error": 0}

# --- 表示用ヘルパー ----------------------------------------------------
def _fmt_issues(issues):
    return ", ".join(issues) if issues else "None"

def _safe_dict(d):
    return d if isinstance(d, dict) else {}

def _log_tool_selection(graph_state, ml_probability):
    gs = _safe_dict(graph_state)
    pre = _safe_dict(gs.get("precheck_hints"))
    ml_cat   = pre.get("ml_category", "-")
    tld_cat  = pre.get("tld_category", "-")
    quick_risk = pre.get("quick_risk", "-")
    known_info = _safe_dict(pre.get("known_domain_info"))
    known_label = known_info.get("label") or known_info.get("kind") or known_info.get("brand")

    selected_tools = gs.get("selected_tools", [])
    flags = _safe_dict(gs.get("tool_execution_flags"))
    llm_used = gs.get("llm_used_selection")
    llm_err  = gs.get("llm_selection_error")

    print("  [ToolSelection]")
    print(f"    ml_probability      : {ml_probability:.3f} (category={ml_cat})")
    print(f"    tld_category        : {tld_cat}  quick_risk={quick_risk}")
    if known_label:
        print(f"    known_domain_info   : {known_label}")
    print(f"    selected_tools      : {selected_tools}")
    print(f"    tool_execution_flags: {flags}")
    print(f"    llm_used_selection  : {llm_used}")
    if llm_err:
        print(f"    llm_selection_error : {llm_err}")

    if llm_used:
        strategy = "llm_structured_output"
    else:
        strategy = "ml_bucket_fallback"
    print(f"    selection_strategy  : {strategy}")

    # フォールバック時は、どのバケットポリシーになったかも表示
    if not llm_used:
        if ml_probability < 0.2:
            policy = "ML<0.2 → brand+cert+domain (3 tools)"
        elif ml_probability < 0.5:
            policy = "0.2≦ML<0.5 → brand+cert+domain (3 tools)"
        else:
            policy = "ML≧0.5 → brand+cert (2 tools)"
        print(f"    selection_policy    : {policy}")

def _log_brand_tool(brand_res):
    br = _safe_dict(brand_res)
    if not br:
        print("  [BrandTool] not executed")
        return

    details = _safe_dict(br.get("details"))
    issues  = br.get("detected_issues") or []
    # ※ brand_impersonation_check の構造に合わせて details 側を優先
    brands  = details.get("detected_brands") or br.get("detected_brands") or []
    used_llm    = details.get("used_llm")
    llm_conf    = details.get("llm_confidence")
    llm_reason  = details.get("llm_reasoning")
    #llm_reason  = (details.get("llm_reasoning") or "")[:160]

    print("  [BrandTool]")
    if br.get("_fallback"):
        print("    ⚠ fallback           : True (brand tool exception or disabled)")
    print(f"    risk_score           : {br.get('risk_score')}")
    print(f"    detected_issues      : {issues}")
    print(f"    detected_brands      : {brands}")
    print(f"    used_llm             : {used_llm}")
    print(f"    llm_confidence       : {llm_conf}")
    print(f"    llm_reasoning (head) : {llm_reason}")
    if used_llm:
        print("    ✅ LLM が実行されました (used_llm=True)")
    if brands:
        print(f"    ✅ ブランド検出があります: {brands}")

def _log_cert_tool(cert_res):
    cr = _safe_dict(cert_res)
    if not cr:
        print("  [CertTool] not executed")
        return
    details = _safe_dict(cr.get("details"))
    print("  [CertTool]")
    if cr.get("_fallback"):
        print("    ⚠ fallback      : True (cert tool exception or disabled)")
    print(f"    risk_score      : {cr.get('risk_score')}")
    print(f"    detected_issues : {cr.get('detected_issues') or []}")
    print(f"    issuer          : {details.get('issuer')}")
    print(f"    is_free_ca      : {details.get('is_free_ca')}")
    print(f"    has_org         : {details.get('has_org')}")
    print(f"    valid_days      : {details.get('valid_days')}")
    print(f"    is_short_term   : {details.get('is_short_term')}")
    print(f"    san_count       : {details.get('san_count')}")
    print(f"    is_self_signed  : {details.get('is_self_signed')}")
    print(f"    is_wildcard     : {details.get('is_wildcard')}")

def _log_domain_tool(domain_res):
    dr = _safe_dict(domain_res)
    if not dr:
        print("  [DomainTool] not executed")
        return
    details = _safe_dict(dr.get("details"))
    print("  [DomainTool]")
    if dr.get("_fallback"):
        print("    ⚠ fallback            : True (domain tool exception or disabled)")
    print(f"    risk_score            : {dr.get('risk_score')}")
    print(f"    detected_issues       : {dr.get('detected_issues') or []}")
    print(f"    base_domain           : {details.get('base_domain')}")
    print(f"    domain_length         : {details.get('domain_length')}"
          f" (category={details.get('domain_length_category')})")
    print(f"    tld / tld_category    : {details.get('tld')} / {details.get('tld_category')}")
    print(f"    entropy               : {details.get('entropy')}")
    print(f"    combo_flags           : {details.get('combo_flags')}")
    legit = _safe_dict(details.get('legitimate_check'))
    if legit:
        print(f"    legitimate_check      : is_legit={legit.get('is_legitimate')}"
              f", brand={legit.get('brand')}, conf={legit.get('confidence')}")

def _log_contextual_tool(ctx_res):
    cx = _safe_dict(ctx_res)
    if not cx:
        print("  [Contextual] not executed")
        return
    details = _safe_dict(cx.get("details"))
    print("  [Contextual]")
    if cx.get("_fallback"):
        print("    ⚠ fallback            : True (contextual tool exception or disabled)")
    print(f"    risk_score            : {cx.get('risk_score')}")
    print(f"    detected_issues       : {cx.get('detected_issues') or []}")
    print(f"    ml_probability        : {details.get('ml_probability')}"
          f" (category={details.get('ml_category')})")
    print(f"    tool_average_risk     : {details.get('tool_average_risk')}")
    print(f"    high_risk_hits        : {details.get('high_risk_hits')}")
    known = _safe_dict(details.get('known_domain'))
    if known:
        print(f"    known_domain          : {known}")
    paradox = _safe_dict(details.get('paradox'))
    if paradox:
        print(f"    paradox_info          : {paradox}")

def _log_final_decision(graph_state, result):
    gs = _safe_dict(graph_state)
    dbg = _safe_dict(gs.get("debug_llm_final"))
    dt_list = gs.get("decision_trace") or []
    print("  [FinalDecision]")
    print(f"    use_llm_decision      : {dbg.get('use_llm_decision')}")
    print(f"    llm_is_none           : {dbg.get('llm_is_none')}")
    print(f"    path                  : {dbg.get('path')}")
    print(f"    success               : {dbg.get('success')}")
    if dbg.get("error"):
        print(f"    error                 : {dbg.get('error')}")

    if dt_list:
        last = _safe_dict(dt_list[-1])
        print(f"    phase6_policy_version : {gs.get('phase6_policy_version', last.get('phase6_version'))}")
        print(f"    ctx_risk_score        : {last.get('ctx_score')}")
        rules = [s.get("rule") for s in last.get("policy_trace", []) if isinstance(s, dict) and "rule" in s]
        notes = [s.get("note") for s in last.get("policy_trace", []) if isinstance(s, dict) and "note" in s]
        if rules:
            print(f"    policy_rules          : {rules}")
        if notes:
            print(f"    policy_notes          : {notes}")

    print(f"    ai_is_phishing        : {result.get('ai_is_phishing')} "
          f"(risk_level={result.get('ai_risk_level')}, conf={result.get('ai_confidence'):.2f})")
    print(f"    final_reasoning(head) : {(result.get('reasoning') or '')[:160]}")

def _log_graph_messages(graph_state, prefix="    "):
    """LangGraph のメッセージストリームから、重要そうなものだけ抜粋表示"""
    gs = _safe_dict(graph_state)
    msgs = gs.get("messages") or []
    if not msgs:
        return
    print("  [GraphMessages] (tool_selection / tool_execution / final_decision only)")
    for m in msgs:
        if isinstance(m, dict):
            content = m.get("content", "")
            role = m.get("role", "msg")
        else:
            content = getattr(m, "content", "")
            role = getattr(m, "role", "msg")
        if not isinstance(content, str):
            content = str(content)
        if any(tag in content for tag in ["[tool_selection]", "[tool_execution]", "[final_decision]"]):
            print(f"{prefix}[{role}] {content}")


# --- ログをCSVに残すための標準出力/標準エラーキャプチャ（print/logging両対応） ---
class _TeeIO:
    def __init__(self, primary, buffer, *, enable_primary: bool = True):
        self.primary = primary
        self.buffer = buffer
        self.enable_primary = bool(enable_primary)
    def write(self, s):
        if self.enable_primary:
            self.primary.write(s)
        self.buffer.write(s)
        return len(s)
    def flush(self):
        if self.enable_primary:
            try:
                self.primary.flush()
            except Exception:
                pass
        try:
            self.buffer.flush()
        except Exception:
            pass

def _json_dumps(obj):
    """CSVに安全に載せるためのJSON化（失敗しても落とさない）"""
    try:
        return json.dumps(obj, ensure_ascii=False, default=str)
    except Exception:
        return json.dumps(str(obj), ensure_ascii=False, default=str)

# --- 各ドメインを評価 ---------------------------------------------------
for idx, row in target_df.iterrows():
    before_len = len(results)  # 追記保存用（このイテレーションで追加された行だけ拾う）
    domain = row['domain']
    ml_prob = row['ml_probability']
    _buf = io.StringIO()
    _tee_out = _TeeIO(sys.stdout, _buf, enable_primary=SHOW_DOMAIN_LOG_ON_SCREEN)
    _tee_err = _TeeIO(sys.stderr, _buf, enable_primary=SHOW_DOMAIN_LOG_ON_SCREEN)
    with redirect_stdout(_tee_out), redirect_stderr(_tee_err):

        try:
            eval_start = time.time()
            result = agent.evaluate(domain, ml_prob)
            elapsed = time.time() - eval_start

            # --- 基本情報 ---
            is_phishing = result.get('ai_is_phishing', False)
            confidence  = result.get('ai_confidence', 0.0)
            risk_level  = result.get('ai_risk_level', 'unknown')

            graph_state = _safe_dict(result.get("graph_state"))
            tool_res = _safe_dict(graph_state.get("tool_results")) or _safe_dict(result.get("tool_results"))

            # Phase4 仕様: brand/cert/domain/contextual_risk_assessment は data 本体が入っている想定
            brand_res  = _safe_dict(tool_res.get('brand'))
            cert_res   = _safe_dict(tool_res.get('cert'))
            domain_res = _safe_dict(tool_res.get('domain'))
            ctx_res    = _safe_dict(tool_res.get('contextual_risk_assessment') or tool_res.get('contextual'))

            # Brand details
            brand_details   = _safe_dict(brand_res.get('details'))
            detected_brands = brand_details.get('detected_brands', [])

            # brand_detected / brand_suspected を tool 側のフラグから取得（brands配列の有無で決めない）
            _b_issues = brand_res.get('detected_issues', []) if isinstance(brand_res, dict) else []
            brand_detected_flag = bool(brand_details.get('brand_detected')) or ('brand_detected' in (_b_issues or []))
            brand_suspected_flag = bool(brand_details.get('brand_suspected')) or ('brand_suspected' in (_b_issues or [])) or ('brand_llm_candidate' in (_b_issues or []))


            # Cert details
            cert_details = _safe_dict(cert_res.get('details'))
            cert_issuer  = cert_details.get('issuer', 'unknown')

            # Domain details
            domain_details = _safe_dict(domain_res.get('details'))
            tld_cat        = domain_details.get('tld_category', '-')

            # Contextual issues
            ctx_issues = ctx_res.get('detected_issues', []) if ctx_res else []

            mark = "🔴" if is_phishing else "🟢"
            print("-" * 80)
            print(f"[{idx+1:3}/{len(target_df)}] {mark} {domain:<35} (ML: {ml_prob:.3f} / Time: {elapsed:.2f}s)")
            print(f"🔍 Domain: {domain} (ml_probability={ml_prob:.3f})")

            # --- Tool Selection 詳細 ---
            _log_tool_selection(graph_state, ml_prob)

            # --- 各ツール詳細 ---
            _log_brand_tool(brand_res)
            _log_cert_tool(cert_res)
            _log_domain_tool(domain_res)
            if ctx_res:
                _log_contextual_tool(ctx_res)

            # --- Final Decision 詳細 ---
            _log_final_decision(graph_state, result)

            # --- LangGraph メッセージ抜粋 ---
            _log_graph_messages(graph_state)

            # --- 保存用データの構築 ---
            dbg_final = _safe_dict(graph_state.get("debug_llm_final"))
            results.append({
                **CODE_FP_ROW,
                'domain': domain,
                'ml_probability': ml_prob,
                'ai_is_phishing': is_phishing,
                'ai_confidence': confidence,
                'ai_risk_level': risk_level,
                'processing_time': elapsed,
                'debug_log': _buf.getvalue(),
                'graph_state_json': _json_dumps(graph_state),
                'tool_results_json': _json_dumps(tool_res),
                # Brand
                'brand_detected': brand_detected_flag,
                'brand_suspected': brand_suspected_flag,
                'brands': detected_brands,
                'brand_used_llm': brand_details.get('used_llm'),
                'brand_llm_quality': brand_details.get('llm_candidate_quality'),
                'brand_llm_evidence_token': brand_details.get('llm_evidence_token'),
                'brand_llm_detected_brand': brand_details.get('llm_detected_brand'),
                'brand_llm_confidence': brand_details.get('llm_confidence'),
                'brand_risk_score': brand_res.get('risk_score'),
                'brand_issues': brand_res.get('detected_issues', []),
                # Cert
                'cert_issues': cert_res.get('detected_issues', []),
                'cert_issuer': cert_issuer,
                'cert_score': cert_res.get('risk_score', 0.0),
                # Domain
                'domain_issues': domain_res.get('detected_issues', []),
                'domain_score': domain_res.get('risk_score', 0.0),
                'tld_category': tld_cat,
                # Contextual
                'ctx_issues': ctx_issues,
                'ctx_score': ctx_res.get('risk_score', None) if ctx_res else None,
                # Tool Selection / Final LLM debug
                'tool_selection_llm_used': graph_state.get('llm_used_selection'),
                'tool_selection_llm_error': graph_state.get('llm_selection_error'),
                'final_llm_path': dbg_final.get('path'),
                'final_llm_success': dbg_final.get('success'),
                'phase6_policy_version': graph_state.get('phase6_policy_version'),
                'module_version': result.get('module_version'),
                'error': None,
            })

        except Exception as e:
            elapsed = time.time() - eval_start
            print(f"[{idx+1:3}/{len(target_df)}] ❌ ERROR: {domain} - {str(e)}")
            results.append({
                **CODE_FP_ROW,
                'domain': domain,
                'ml_probability': ml_prob,
                'ai_is_phishing': False,
                'ai_confidence': 0.0,
                'ai_risk_level': 'error',
                'processing_time': elapsed,
                'debug_log': _buf.getvalue(),
                'graph_state_json': _json_dumps({}),
                'tool_results_json': _json_dumps({}),
                'error': str(e),
            })

    # --- 追記保存（500件ごと）: このドメインの結果（results[-1]）をバッファに積み、必要なら追記 ---
    try:
        if len(results) > before_len:
            _chunk_buf.append(results[-1])
            _flush_chunk_if_needed(force=False)
    except Exception:
        # 追記保存の失敗で評価自体を止めない（ログは結果側に残す）
        pass
    # --- 画面表示（コンパクト進捗） ---------------------------------------
    # NOTE: 詳細ログは _buf に全量保存しているので、画面側は軽くする
    try:
        _last = results[-1] if results else None
    except Exception:
        _last = None

    if _last is not None:
        if _last.get("error"):
            _screen_counts["error"] += 1
        else:
            if bool(_last.get("ai_is_phishing")):
                _screen_counts["phish"] += 1
            else:
                _screen_counts["benign"] += 1

        # 詳細ログを画面に出さないモードのときだけ、進捗を間引いて表示
        if (not SHOW_DOMAIN_LOG_ON_SCREEN) and (
            ((idx + 1) % PROGRESS_EVERY == 0) or (idx == (len(target_df) - 1)) or _last.get("error")
        ):
            if _HAS_CLEAR_OUTPUT and _USE_CLEAR_OUTPUT:
                try:
                    clear_output(wait=True)
                except Exception:
                    pass

            elapsed_total = time.time() - start_time
            mark = "❌" if _last.get("error") else ("🔴" if _last.get("ai_is_phishing") else "🟢")
            dom  = _last.get("domain", "-")
            mlp  = float(_last.get("ml_probability") or 0.0)
            conf = float(_last.get("ai_confidence") or 0.0)
            rl   = _last.get("ai_risk_level", "-")
            tsec = float(_last.get("processing_time") or 0.0)

            print(f"[PROGRESS] {idx+1}/{len(target_df)}  phishing={_screen_counts['phish']}  benign={_screen_counts['benign']}  error={_screen_counts['error']}  elapsed={elapsed_total:.1f}s")
            print(f"          last: {mark} {dom} (ML={mlp:.3f} risk={rl} conf={conf:.2f} t={tsec:.2f}s)")
            if _last.get("error"):
                print(f"          last_error: {_last.get('error')}")

# --- 完了処理 ------------------------------------------------------------
total_time = time.time() - start_time
print("=" * 80)
print(f"[INFO] Full evaluation complete! Total time: {total_time:.2f}s")

# DataFrame化（後続セルの分析用にメモリ上にも保持）
results_df = pd.DataFrame(results)

# --- 追記保存: 端数（<500件）も必ず保存して終了 ---
_flush_chunk_if_needed(force=True)

print(f"[INFO] Results saved (append) to: {full_eval_path}  rows_written={_written_rows}")


In [ ]:
def debug_one_domain(domain: str, ml: float = 0.3):
    res = agent.evaluate(domain, ml)
    gs = (res.get("graph_state") or {})
    dbg = (gs.get("debug_llm_final") or {})
    asmt = gs.get("final_assessment")

    print("=== RAW RESULT ===")
    print(f"domain         : {res.get('domain')}")
    print(f"ml_probability : {res.get('ml_probability')}")
    print(f"success        : {res.get('success')}")
    print(f"ai_is_phishing : {res.get('ai_is_phishing')}")
    print(f"ai_confidence  : {res.get('ai_confidence')}")
    print(f"ai_risk_level  : {res.get('ai_risk_level')}")
    print(f"reasoning      : {res.get('reasoning')[:120]!r}...")

    print("\n=== LLM FINAL DEBUG (debug_llm_final) ===")
    print("  path           :", dbg.get("path"))
    print("  success        :", dbg.get("success"))
    print("  error          :", dbg.get("error"))
    print("  exception_type :", dbg.get("exception_type"))
    print("  use_llm_decision:", dbg.get("use_llm_decision"))
    print("  llm_is_none    :", dbg.get("llm_is_none"))
    print("  strict_mode    :", dbg.get("strict_mode"))
    print("  fallback_reason:", dbg.get("fallback_reason"))

    print("\n=== FINAL ASSESSMENT OBJECT ===")
    print("  type(asmt):", type(asmt))
    if asmt is not None:
        print("  is_phishing :", getattr(asmt, 'is_phishing', None))
        print("  confidence  :", getattr(asmt, 'confidence', None))
        print("  risk_level  :", getattr(asmt, 'risk_level', None))

    print("\n=== FALLBACK LOCATIONS ===")
    print("  ", gs.get("fallback_locations"))

    return res, gs, dbg


In [ ]:
# ============================================
# Cell 6: graph.stream を使った単発デバッグ
# ============================================
from copy import deepcopy

# ★ どの行をデバッグするか指定（例: 0番目 = 1件目）
debug_idx = 0

# guard: target_df が空なら原因（サンプリング設定/キャッシュ）を先に確認
if target_df is None or len(target_df) == 0:
    raise ValueError(
        "target_df is empty. N_SAMPLE=0/-1(ALL) の場合は、既存CSVキャッシュを無視して pickle から再生成する必要があります。\n"
        "→ Cell0/Cell3 を上から再実行してください。"
    )

row = target_df.iloc[debug_idx]
domain = row["domain"]
ml_prob = float(row["ml_probability"])
print(f"[DEBUG] domain={domain}, ml_probability={ml_prob:.4f}")

# --- evaluate() が内部で作る initial_state とほぼ同じものを用意 ---
initial_state = {
    "domain": domain,
    "ml_probability": float(ml_prob or 0.0),
    "strict_mode": agent.strict_mode,
    "current_step": "initial",
    "precheck_hints": {},
    "selected_tools": [],
    "tool_results": {},
    "final_assessment": None,
    "error": None,
    "retry_count": 0,
    "fallback_count": 0,
    "fallback_locations": [],
    "tool_execution_flags": {},
    "next_step": "",
    "messages": [],  # AgentState に追加した debug 用チャンネル
}

if agent.graph is None:
    raise RuntimeError("agent.graph が None なので stream() が使えません")

final_state = None
for step in agent.graph.stream(initial_state, stream_mode="values"):
    print(f"\n[STEP] current_step={step.get('current_step')}")
    msgs = step.get("messages") or []
    if not msgs:
        continue

    last = msgs[-1]

    # メッセージ本文の取り出し
    if hasattr(last, "content"):
        text = last.content if isinstance(last.content, str) else str(last.content)
    elif isinstance(last, dict):
        text = str(last.get("content", last))
    else:
        text = str(last)

    # precheck / tool_selection / tool_execution だけ表示
    if any(tag in text for tag in ("[precheck]", "[tool_selection]", "[tool_execution]")):
        if hasattr(last, "pretty_print"):
            last.pretty_print()
        else:
            print("    " + text)

    final_state = step

# --- 最終状態のざっくり確認（余裕があれば）---
if final_state is not None:
    fa = final_state.get("final_assessment")
    print("\n[FINAL STATE]")
    print("  is_phishing :", getattr(fa, "is_phishing", None))
    print("  confidence  :", getattr(fa, "confidence", None))
    print("  risk_level  :", getattr(fa, "risk_level", None))
    print("  tool_keys   :", list((final_state.get("tool_results") or {}).keys()))
else:
    print("[DEBUG] final_state が取得できませんでした")


In [ ]:
# ============================================
# Cell 6: ツール別詳細統計サマリー
# ============================================
import numpy as np

# データの準備（エラーなしのデータのみ）
valid_df = results_df[results_df['error'].isna()].copy()

print("\n" + "=" * 80)
print("🔍 DETAILED TOOL ANALYSIS SUMMARY")
print("=" * 80)

# 1. 全体統計
phish_count = valid_df['ai_is_phishing'].sum()
total = len(valid_df)
print(f"\n📊 Overall Performance")
print(f"  Total Domains: {total}")
print(f"  Phishing Detected: {phish_count} ({phish_count/total*100:.1f}%)")
print(f"  Avg Processing Time: {valid_df['processing_time'].mean():.2f}s")

# 2. Certificate Analysis 統計 (今回注目の機能)
print(f"\n🔒 Certificate Analysis Stats")
# Issueごとのカウント
all_cert_issues = []
for issues in valid_df['cert_issues']:
    if isinstance(issues, list):
        all_cert_issues.extend(issues)
    elif isinstance(issues, str) and issues: # 文字列の場合のケア
        import ast
        try:
            all_cert_issues.extend(ast.literal_eval(issues))
        except:
            pass

if all_cert_issues:
    from collections import Counter
    cert_counts = Counter(all_cert_issues)
    for issue, count in cert_counts.most_common():
        print(f"  - {issue:20}: {count} domains")
else:
    print("  No certificate issues detected.")

# 発行者(Issuer)のTop5 (Free CAの確認など)
issuers = valid_df[valid_df['cert_issuer'] != 'unknown']['cert_issuer']
if not issuers.empty:
    print(f"  [Top Issuers]")
    print(issuers.value_counts().head(5).to_string(header=False))

# 3. Domain Analysis 統計
print(f"\n🌐 Domain Analysis Stats")
all_domain_issues = []
for issues in valid_df['domain_issues']:
    if isinstance(issues, list):
        all_domain_issues.extend(issues)

if all_domain_issues:
    dom_counts = Counter(all_domain_issues)
    for issue, count in dom_counts.most_common():
        print(f"  - {issue:20}: {count} domains")

# 4. Brand Analysis 統計
print(f"\n🏷️  Brand Analysis Stats")
brand_hits = valid_df[valid_df['brand_detected'] == True]
print(f"  Brand Detected: {len(brand_hits)} domains")
if not brand_hits.empty:
    all_brands = []
    for brands in brand_hits['brands']:
        if isinstance(brands, list): all_brands.extend(brands)
    print(f"  [Top Detected Brands]")
    print(pd.Series(all_brands).value_counts().head(5).to_string(header=False))

print("\n" + "=" * 80)
print(f"✅ Verification Complete. Check CSV for per-domain details.")

In [ ]:
# === セル1: パス設定 & test_data / モデル読み込み（artifacts/<RUN_ID>/... に統一） ===
import os
from pathlib import Path

import numpy as np
import pandas as pd
import joblib
import xgboost as xgb

# Cell 1 で定義された PROCESSED_DIR / MODELS_DIR / RUN_ID を利用
if "PROCESSED_DIR" not in globals() or "MODELS_DIR" not in globals():
    raise RuntimeError("PROCESSED_DIR / MODELS_DIR が未定義です。先に Cell 1 を実行してください。")

DATA_DIR = Path(PROCESSED_DIR)
MODELS_DIR = Path(MODELS_DIR)  # すでに Path のはずだが念のため

# 02_xgboost_training_evaluation_... の新仕様に合わせて:
#   - test_data.pkl: artifacts/<RUN_ID>/processed/test_data.pkl
#   - xgboost_model.pkl, scaler.pkl: artifacts/<RUN_ID>/models/...
TEST_DATA_PKL = DATA_DIR / "test_data.pkl"
XGB_MODEL_PKL = MODELS_DIR / "xgboost_model.pkl"
SCALER_PKL    = MODELS_DIR / "scaler.pkl"

print("TEST_DATA_PKL :", TEST_DATA_PKL)
print("XGB_MODEL_PKL :", XGB_MODEL_PKL)
print("SCALER_PKL    :", SCALER_PKL)

In [ ]:
# === セル2: test_data + XGBoost で ml_probability を再計算 ===

# 02 で joblib.dump していれば joblib.load でそのまま読めます
test_data = joblib.load(TEST_DATA_PKL)

X_test = np.asarray(test_data["X"])
y_test = np.asarray(test_data["y"]).astype(int)
domains_test = np.asarray(test_data["domains"])

xgb_model = joblib.load(XGB_MODEL_PKL)
try:
    scaler = joblib.load(SCALER_PKL)
except FileNotFoundError:
    scaler = None
    print("⚠ scaler.pkl が見つからなかったので、スケーリング無しで推論します。")

def compute_ml_probabilities(model, X, scaler=None):
    """XGBoost モデルから P(phish) を推定するヘルパー"""
    X_in = scaler.transform(X) if scaler is not None else X

    # sklearn API (XGBClassifier) の場合
    if hasattr(model, "predict_proba"):
        proba = np.asarray(model.predict_proba(X_in))
        if proba.ndim == 2 and proba.shape[1] >= 2:
            return proba[:, 1]
        return proba.ravel()

    # Booster API の場合
    if hasattr(model, "predict"):
        dtest = xgb.DMatrix(X_in)
        proba = np.asarray(model.predict(dtest))
        return proba.ravel()

    raise RuntimeError("予期しない XGBoost モデル型です (predict_proba/predict が見つからない)。")

ml_probs_test = compute_ml_probabilities(xgb_model, X_test, scaler=scaler)

test_df = pd.DataFrame({
    "domain": domains_test,
    "label": y_test,
    "ml_probability": ml_probs_test,
})
test_df.head()


In [ ]:
# === セル3: 正常 (label==0) から N_BENIGN_SAMPLE 件サンプリング ===
# [ChangeLog] 2025-12-15: N_BENIGN_SAMPLE<=0(ALL) の場合は全件を使う

# N_BENIGN_SAMPLE は Cell 1 の設定に従う（必要なら環境変数 N_BENIGN_SAMPLE で上書き）
# RANDOM_STATE は Cell 1 で定義（環境変数 RANDOM_STATE でも上書き可）

benign_candidates = test_df.query("label == 0 and ml_probability < 0.5").copy()
print("benign candidates (label==0 & ml_prob<0.5):", len(benign_candidates))

# N_BENIGN_SAMPLE <= 0 の場合は「全件処理」
if (N_BENIGN_SAMPLE <= 0) or (len(benign_candidates) <= N_BENIGN_SAMPLE):
    print(f"候補が {len(benign_candidates)} 件なので、そのまま全件使います。")
    benign_sample_df = benign_candidates.reset_index(drop=True)
else:
    benign_sample_df = benign_candidates.sample(N_BENIGN_SAMPLE, random_state=RANDOM_STATE).reset_index(drop=True)

benign_sample_df.head()


In [ ]:
# === セル4: benign サンプルに LangGraphPhishingAgent を実行 ===
# ここでは agent は既に前のセルで初期化されている前提

benign_agent_results = []
for row in benign_sample_df.itertuples(index=False):
    res = agent.evaluate(
        domain=row.domain,
        ml_probability=float(row.ml_probability),
        # external_data は __init__ で渡しているなら省略でOK
    )
    # [ChangeLog] 2025-12-17: Attach code fingerprint columns to each record.
    if isinstance(res, dict) and 'CODE_FP_ROW' in globals():
        res.update(CODE_FP_ROW)
    benign_agent_results.append(res)

benign_agent_df = pd.DataFrame(benign_agent_results)

# 必要な列だけをマージ
benign_eval_df = benign_sample_df.merge(
    benign_agent_df[["domain", "ai_is_phishing", "ai_confidence", "ai_risk_level"]],
    on="domain",
    how="left",
)
benign_eval_df["label"] = 0  # ground truth
benign_eval_df.head()


In [ ]:
# === 追加セル: benign 評価結果をCSVへ保存（共有用） ===
import datetime
from pathlib import Path

ts = datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S')
eval_id = None
if "CODE_FP_ROW" in globals() and isinstance(globals().get("CODE_FP_ROW"), dict):
    eval_id = globals()["CODE_FP_ROW"].get("eval_id")
if not eval_id:
    eval_id = ts

# 1) ラベル付き簡易サマリ
out_dir = Path(LOGS_DIR)
out_dir.mkdir(parents=True, exist_ok=True)

eval_path = out_dir / f"benign{len(benign_eval_df)}_eval__evalid_{eval_id}__ts_{ts}.csv"
benign_eval_df.to_csv(eval_path, index=False)

# 2) フルログ（agent.evaluate の生出力を含む）
full_path = out_dir / f"benign{len(benign_agent_df)}_full_eval__evalid_{eval_id}__ts_{ts}.csv"
benign_agent_df.to_csv(full_path, index=False)

print('saved:', eval_path)
print('saved:', full_path)


In [ ]:
# === 追加セル: hard-negative benign を作成（MLが高い正常ドメイン） ===
# 目的: 「MLが高い正常」を集めて、FPが増えないか検証する（hard negatives）
# デフォルトは 200 件。必要なら環境変数 N_BENIGN_HARD_SAMPLE で上書きできます。
# [ChangeLog] 2025-12-15: N_BENIGN_HARD_SAMPLE<=0(ALL) の場合は全件を使う

import os
import numpy as np

N_BENIGN_HARD_SAMPLE = int(os.getenv("N_BENIGN_HARD_SAMPLE", "200"))

# label==0 全体から ML の高い順に並べる
benign_all = test_df.query("label == 0").copy()
benign_all = benign_all.sort_values("ml_probability", ascending=False)

# すでに benign_sample_df を作っている場合は重複を避ける（任意）
try:
    used = set(benign_sample_df["domain"].astype(str))
    benign_all = benign_all[~benign_all["domain"].astype(str).isin(used)]
    print("removed overlap with benign_sample_df:", len(used), "used")
except Exception:
    pass

# 全件処理（N<=0） or 候補不足なら全件
if (N_BENIGN_HARD_SAMPLE <= 0) or (len(benign_all) <= N_BENIGN_HARD_SAMPLE):
    print(f"hard benign candidates are only {len(benign_all)}; using all.")
    benign_hard_sample_df = benign_all
else:
    benign_hard_sample_df = benign_all.head(N_BENIGN_HARD_SAMPLE)

benign_hard_sample_df = benign_hard_sample_df.reset_index(drop=True)

print("hard benign sample size:", len(benign_hard_sample_df))
if len(benign_hard_sample_df):
    print("ml_probability (hard benign) min/mean/max:",
          float(benign_hard_sample_df["ml_probability"].min()),
          float(benign_hard_sample_df["ml_probability"].mean()),
          float(benign_hard_sample_df["ml_probability"].max()))
benign_hard_sample_df.head()


In [ ]:
# === 追加セル: hard-negative benign を評価（full_eval を作る） ===
# ここでも agent は既に初期化済みの前提です。

benign_hard_agent_results = []
for row in benign_hard_sample_df.itertuples(index=False):
    res = agent.evaluate(
        domain=row.domain,
        ml_probability=float(row.ml_probability),
    )
    # [ChangeLog] 2025-12-17: Attach code fingerprint columns to each record.
    if isinstance(res, dict) and 'CODE_FP_ROW' in globals():
        res.update(CODE_FP_ROW)
    benign_hard_agent_results.append(res)

benign_hard_agent_df = pd.DataFrame(benign_hard_agent_results)

# 必要な列だけをマージ（label=0）
benign_hard_eval_df = benign_hard_sample_df.merge(
    benign_hard_agent_df[["domain", "ai_is_phishing", "ai_confidence", "ai_risk_level"]],
    on="domain",
    how="left",
)
benign_hard_eval_df["label"] = 0  # ground truth
benign_hard_eval_df.head()


In [ ]:
# === 追加セル: benign_hard 評価結果をCSVへ保存（共有用） ===
import datetime
from pathlib import Path

ts = datetime.datetime.now().strftime('%Y-%m-%d_%H%M%S')
eval_id = None
if "CODE_FP_ROW" in globals() and isinstance(globals().get("CODE_FP_ROW"), dict):
    eval_id = globals()["CODE_FP_ROW"].get("eval_id")
if not eval_id:
    eval_id = ts

# 1) ラベル付き簡易サマリ
out_dir = Path(LOGS_DIR)
out_dir.mkdir(parents=True, exist_ok=True)

eval_path = out_dir / f"benign_hard{len(benign_hard_eval_df)}_eval__evalid_{eval_id}__ts_{ts}.csv"
benign_hard_eval_df.to_csv(eval_path, index=False)

# 2) フルログ（agent.evaluate の生出力を含む）
full_path = out_dir / f"benign_hard{len(benign_hard_agent_df)}_full_eval__evalid_{eval_id}__ts_{ts}.csv"
benign_hard_agent_df.to_csv(full_path, index=False)

print('saved:', eval_path)
print('saved:', full_path)


In [ ]:
# === セル5: FN サンプル側の整形 (全部 label=1) ===

# target_df: FN サンプル（n=len(target_df), 全部 phish想定）
# results_df: target_df に対して agent.evaluate() 済みの DataFrame
fn_eval_df = (
    target_df[["domain", "ml_probability"]]
    .merge(
        results_df[["domain", "ai_is_phishing", "ai_confidence", "ai_risk_level"]],
        on="domain",
        how="left",
    )
)
fn_eval_df["label"] = 1  # ground truth = phishing
fn_eval_df.head()


In [ ]:
# === セル6: phishN + benignN を結合 ===

eval_df = pd.concat(
    [fn_eval_df, benign_eval_df],
    ignore_index=True,
)

print(eval_df[["label", "ai_is_phishing"]].value_counts())
eval_df.head()


In [ ]:
# === セル7: Agent 単体の混同行列と各種指標を計算 ===
from dataclasses import dataclass

@dataclass
class ConfusionResult:
    TP: int
    FP: int
    TN: int
    FN: int
    precision: float
    recall: float
    f1: float
    fbeta: float
    fpr: float

def compute_confusion_and_scores(
    df: pd.DataFrame,
    label_col: str = "label",
    pred_col: str = "ai_is_phishing",
    beta: float = 2.0,
) -> ConfusionResult:
    y_true = df[label_col].astype(int).to_numpy()
    y_pred = df[pred_col].astype(int).to_numpy()

    tp = int(((y_true == 1) & (y_pred == 1)).sum())
    fp = int(((y_true == 0) & (y_pred == 1)).sum())
    tn = int(((y_true == 0) & (y_pred == 0)).sum())
    fn = int(((y_true == 1) & (y_pred == 0)).sum())

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    beta2 = beta ** 2
    denom = beta2 * precision + recall
    fbeta = ((1 + beta2) * precision * recall) / denom if denom > 0 else 0.0

    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0.0

    return ConfusionResult(tp, fp, tn, fn, precision, recall, f1, fbeta, fpr)

metrics_agent = compute_confusion_and_scores(eval_df, beta=2.0)
metrics_agent


In [ ]:
# いまのままでもOKな FP/FN の抽出
is_tp = (eval_df["label"] == 1) & (eval_df["ai_is_phishing"] == 1)
is_fp = (eval_df["label"] == 0) & (eval_df["ai_is_phishing"] == 1)
is_tn = (eval_df["label"] == 0) & (eval_df["ai_is_phishing"] == 0)
is_fn = (eval_df["label"] == 1) & (eval_df["ai_is_phishing"] == 0)

tp_df = eval_df[is_tp].copy()
fp_df = eval_df[is_fp].copy()
tn_df = eval_df[is_tn].copy()
fn_df = eval_df[is_fn].copy()


In [ ]:
# 候補にする列（環境に合わせて増減OK）
candidate_cols = [
    "domain",
    "label",
    "ai_is_phishing",
    "ml_probability",
    "ai_confidence",
    "ai_risk_level",
    "error",
    "phase",
    "graph_state",
    # "tools_used",  # ← 今は無いのでコメントアウト
]

# 実際に存在する列だけを使う
fp_cols = [c for c in candidate_cols if c in fp_df.columns]
fn_cols = [c for c in candidate_cols if c in fn_df.columns]

print("FP columns:", fp_cols)
display(fp_df[fp_cols].head(20))

print("FN columns:", fn_cols)
display(fn_df[fn_cols].head(20))


In [ ]:
import pandas as pd

# 列を省略せず表示
pd.set_option("display.max_columns", None)
# 横幅を広げる（好みで調整）
pd.set_option("display.width", 200)
# 長い文字列も省略しない
pd.set_option("display.max_colwidth", None)


In [ ]:
# FP の全列・全行
#display(fp_df)

# FN の全列・全行
display(fn_df)


In [ ]:
# =========================
# FP / FN 分析CSVの生成（eval_id 固定・参照元固定版）
#   - 500件ごとに「追記」保存（分割ファイルを作らない）
#   - 端数も捨てずに保存
#   - 進捗表示あり
#   - 一時停止: LOGS_DIR/_PAUSE_FP_FN_EXPORT が存在すると次chunk手前で停止
#   - 再開: チェックポイント（.checkpoint.json）で重複追記を防止
# =========================
from pathlib import Path
import datetime
import pandas as pd
import numpy as np
import time
import json

print("=== Export FP/FN analysis CSVs (append mode, eval_id-fixed) ===")

CHUNK_SIZE = 500

# 追跡用の指紋列（存在する分だけ使う）
FINGERPRINT_COLS = [
    "eval_id",
    "phase6_policy_version_code",
    "phase6_wiring_file",
    "phase6_wiring_sha256",
    "llm_final_decision_file",
    "llm_final_decision_sha256",
    "langgraph_module_file",
    "langgraph_module_sha256",
    "dual_import_langgraph_module",
]

BASE_CASE_COLS = [
    "domain",
    "label",
    "ml_probability",
    "ai_is_phishing",
    "ai_confidence",
    "ai_risk_level",
]

def _infer_eval_id(*dfs) -> str:
    # 1) CODE_FP_ROW を最優先（Notebook実行単位で固定される想定）
    if "CODE_FP_ROW" in globals() and isinstance(globals().get("CODE_FP_ROW"), dict):
        _eid = globals()["CODE_FP_ROW"].get("eval_id")
        if _eid:
            return str(_eid)

    # 2) 各DFの eval_id が単一ならそれを採用
    for df in dfs:
        if isinstance(df, pd.DataFrame) and "eval_id" in df.columns:
            vals = df["eval_id"].dropna().astype(str).unique()
            if len(vals) == 1:
                return vals[0]

    # 3) 最後の手段
    return datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")


def _existing(cols, df):
    return [c for c in cols if isinstance(df, pd.DataFrame) and c in df.columns]


def _load_ckpt(path: Path) -> int:
    """checkpoint から next_start を読む（無ければ0）。"""
    try:
        if path.exists():
            obj = json.loads(path.read_text(encoding="utf-8"))
            n = int(obj.get("next_start", 0))
            return max(0, n)
    except Exception:
        return 0
    return 0


def _save_ckpt(path: Path, next_start: int) -> None:
    try:
        path.write_text(json.dumps({"next_start": int(next_start)}, ensure_ascii=False), encoding="utf-8")
    except Exception:
        pass


def _append_df_in_chunks(df_in: pd.DataFrame, out_path: Path, ckpt_path: Path, pause_path: Path, tag: str):
    """
    df_in を out_path に 500行ごと追記する。
    - ckpt_path の next_start から再開（重複追記防止）
    - pause_path があれば次chunk手前で停止（端数は捨てない）
    """
    total = int(len(df_in))
    start = _load_ckpt(ckpt_path)

    # 既存CSVが無いのに start>0 は矛盾 → 0に戻す
    if start > 0 and (not out_path.exists()):
        start = 0
        _save_ckpt(ckpt_path, 0)

    # ヘッダの有無: ファイルが存在してサイズ>0なら header=False
    wrote_any = out_path.exists() and (out_path.stat().st_size > 0)

    if total == 0:
        # 空でもヘッダだけ作っておく（後工程の存在確認が楽）
        if not wrote_any:
            df_in.head(0).to_csv(out_path, index=False)
        return {"path": str(out_path), "total": total, "written": 0, "start": start, "paused": False}

    if start >= total:
        print(f"[EXPORT:{tag}] already complete: {total}/{total} -> {out_path.name}")
        return {"path": str(out_path), "total": total, "written": 0, "start": start, "paused": False}

    t0 = time.time()
    written = 0
    paused = False

    for s in range(start, total, CHUNK_SIZE):
        if pause_path.exists():
            print(f"[PAUSE] {pause_path} exists. Stop before writing chunk {s}:{min(s+CHUNK_SIZE,total)} for {out_path.name}")
            paused = True
            break

        e = min(s + CHUNK_SIZE, total)
        chunk_df = df_in.iloc[s:e]
        chunk_df.to_csv(out_path, mode="a", header=(not wrote_any), index=False)
        wrote_any = True
        written += int(len(chunk_df))
        _save_ckpt(ckpt_path, e)

        elapsed = time.time() - t0
        print(f"[EXPORT:{tag}] {e}/{total} appended -> {out_path.name}  (elapsed={elapsed:.1f}s)")

    return {"path": str(out_path), "total": total, "written": written, "start": start, "paused": paused}


def _export_cases_and_details(*, full_df: pd.DataFrame, label: int, dataset_tag: str, out_dir: Path, pause_path: Path) -> dict:
    """
    full_df（＝その実行で作ったfull_eval DF）だけを参照して FN/FP を作り、500件ごとに追記保存する。

    - label=1: FN（真:phish なのに AIが benign と判定）
    - label=0: FP（真:benign なのに AIが phish と判定）

    重要:
      - 分割ファイルは作らない（結合ミス防止）
      - debug_log 等の情報量は削らない（detailsは該当行を丸ごと）
      - 再開は checkpoint で制御（CSVの改行混入があるので行数カウントは危険）
    """
    assert isinstance(full_df, pd.DataFrame)

    eval_id = _infer_eval_id(full_df)
    fp_cols = _existing(FINGERPRINT_COLS, full_df)

    df = full_df.copy()
    df["label"] = int(label)

    if int(label) == 1:
        # FN: phishing(1) なのに ai_is_phishing=False
        kind = "fn"
        cases_mask = (df["ai_is_phishing"] == False)
    else:
        # FP: benign(0) なのに ai_is_phishing=True
        kind = "fp"
        cases_mask = (df["ai_is_phishing"] == True)

    # cases: 軽量（分析軸 + fingerprint）
    cases_cols = _existing(BASE_CASE_COLS, df) + fp_cols
    cases_df = df.loc[cases_mask, cases_cols].copy()

    # details: 情報量を落とさない（該当行を丸ごと）
    details_df = df.loc[cases_mask].copy()

    n_total = int(len(df))
    n_cases = int(len(cases_df))

    cases_path = out_dir / f"{kind}_cases_full_for_analysis__{dataset_tag}__evalid_{eval_id}.csv"
    details_path = out_dir / f"{kind}_details_full_for_analysis__{dataset_tag}__evalid_{eval_id}.csv"

    cases_ckpt = out_dir / f"{cases_path.name}.checkpoint.json"
    details_ckpt = out_dir / f"{details_path.name}.checkpoint.json"

    print("-" * 80)
    print(f"[EXPORT] {kind.upper()} {dataset_tag}  cases={n_cases}/{n_total}  chunk={CHUNK_SIZE}")

    cases_info = _append_df_in_chunks(cases_df, cases_path, cases_ckpt, pause_path, f"{kind}-cases:{dataset_tag}")
    details_info = _append_df_in_chunks(details_df, details_path, details_ckpt, pause_path, f"{kind}-details:{dataset_tag}")

    return {
        "dataset": dataset_tag,
        "label": int(label),
        "kind": kind,
        "eval_id": eval_id,
        "n_total": n_total,
        "n_cases": n_cases,
        "cases_path": str(cases_path),
        "details_path": str(details_path),
        "cases_checkpoint": str(cases_ckpt),
        "details_checkpoint": str(details_ckpt),
        "paused": bool(cases_info.get("paused") or details_info.get("paused")),
    }


# 出力先（要件: run_id/logs 配下へ固定）
if "LOGS_DIR" not in globals() or not globals().get("LOGS_DIR"):
    raise RuntimeError("LOGS_DIR is not set. Run Cell 1 first.")
out_dir = Path(globals()["LOGS_DIR"]).resolve()
out_dir.mkdir(parents=True, exist_ok=True)

# 一時停止ファイル（ここにファイルがあると次chunk手前で停止）
pause_path = out_dir / "_PAUSE_FP_FN_EXPORT"

manifest = []

# --- random（phish想定）: results_df が full_eval ---
if "results_df" in globals() and isinstance(globals().get("results_df"), pd.DataFrame):
    manifest.append(
        _export_cases_and_details(
            full_df=globals()["results_df"],
            label=1,
            dataset_tag=f"random{len(globals()['results_df'])}",
            out_dir=out_dir,
            pause_path=pause_path,
        )
    )
else:
    print("  (skip) results_df not found -> random FN export skipped")

# --- benign: benign_agent_df が full_eval ---
if "benign_agent_df" in globals() and isinstance(globals().get("benign_agent_df"), pd.DataFrame):
    manifest.append(
        _export_cases_and_details(
            full_df=globals()["benign_agent_df"],
            label=0,
            dataset_tag=f"benign{len(globals()['benign_agent_df'])}",
            out_dir=out_dir,
            pause_path=pause_path,
        )
    )
else:
    print("  (skip) benign_agent_df not found -> benign FP export skipped")

# --- benign_hard ---
if "benign_hard_agent_df" in globals() and isinstance(globals().get("benign_hard_agent_df"), pd.DataFrame):
    manifest.append(
        _export_cases_and_details(
            full_df=globals()["benign_hard_agent_df"],
            label=0,
            dataset_tag=f"benign_hard{len(globals()['benign_hard_agent_df'])}",
            out_dir=out_dir,
            pause_path=pause_path,
        )
    )
else:
    print("  (skip) benign_hard_agent_df not found -> hard FP export skipped")

# 念のため（None混入事故防止）
manifest = [m for m in manifest if isinstance(m, dict)]

manifest_df = pd.DataFrame(manifest)
manifest_eval_id = _infer_eval_id(*(globals().get(n) for n in ["results_df", "benign_agent_df", "benign_hard_agent_df"]))
manifest_path = out_dir / f"analysis_manifest__evalid_{manifest_eval_id}.csv"
manifest_df.to_csv(manifest_path, index=False)

print("\nSaved analysis CSVs to:")
for r in manifest:
    print(f"  - {str(r['kind']).upper()} {r['dataset']}: {r['n_cases']}/{r['n_total']}  paused={r.get('paused')}")
    print(f"      cases      : {r['cases_path']}")
    print(f"      details    : {r['details_path']}")
    print(f"      cases_ckpt : {r['cases_checkpoint']}")
    print(f"      details_ckpt: {r['details_checkpoint']}")
print(f"  - manifest : {manifest_path}")


In [ ]:
# =========================
# Preview: FP / FN cases（上位だけ表示）
# =========================
from IPython.display import display
import pandas as pd

print("=== Preview (top cases) ===")

def _preview(df, title, n=10, sort_col=None, ascending=False):
    if df is None or not isinstance(df, pd.DataFrame) or len(df)==0:
        print(f"  (none) {title}")
        return
    _df=df.copy()
    if sort_col and sort_col in _df.columns:
        _df=_df.sort_values(sort_col, ascending=ascending)
    display(_df.head(n))

# random FN
if "results_df" in globals() and isinstance(globals().get("results_df"), pd.DataFrame):
    fn_df_preview = globals()["results_df"][globals()["results_df"]["ai_is_phishing"]==False][
        [c for c in ["domain","ml_probability","ai_is_phishing","ai_confidence","ai_risk_level"] if c in globals()["results_df"].columns]
    ]
    print(f"random FN: {len(fn_df_preview)}/{len(globals()['results_df'])}")
    _preview(fn_df_preview, "random FN (ml_probability desc)", n=15, sort_col="ml_probability", ascending=False)

# benign FP
if "benign_agent_df" in globals() and isinstance(globals().get("benign_agent_df"), pd.DataFrame):
    fp_df_preview = globals()["benign_agent_df"][globals()["benign_agent_df"]["ai_is_phishing"]==True][
        [c for c in ["domain","ml_probability","ai_is_phishing","ai_confidence","ai_risk_level","detected_brands"] if c in globals()["benign_agent_df"].columns]
    ]
    print(f"benign FP: {len(fp_df_preview)}/{len(globals()['benign_agent_df'])}")
    _preview(fp_df_preview, "benign FP (ml_probability desc)", n=20, sort_col="ml_probability", ascending=False)

# benign_hard FP
if "benign_hard_agent_df" in globals() and isinstance(globals().get("benign_hard_agent_df"), pd.DataFrame):
    fp_hard_preview = globals()["benign_hard_agent_df"][globals()["benign_hard_agent_df"]["ai_is_phishing"]==True][
        [c for c in ["domain","ml_probability","ai_is_phishing","ai_confidence","ai_risk_level","detected_brands"] if c in globals()["benign_hard_agent_df"].columns]
    ]
    print(f"benign_hard FP: {len(fp_hard_preview)}/{len(globals()['benign_hard_agent_df'])}")
    _preview(fp_hard_preview, "benign_hard FP (ml_probability desc)", n=20, sort_col="ml_probability", ascending=False)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# === ConfusionResult から混同行列を作成 ===
cm = np.array([
    [metrics_agent.TN, metrics_agent.FP],
    [metrics_agent.FN, metrics_agent.TP],
])

labels = ["benign (0)", "phish (1)"]

# 可視化したい指標（必要に応じて増減させてください）
score_names = ["precision", "recall", "f1", "fbeta", "fpr"]
score_values = [
    metrics_agent.precision,
    metrics_agent.recall,
    metrics_agent.f1,
    metrics_agent.fbeta,
    metrics_agent.fpr,
]

# === 描画 ===
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 1) 混同行列のヒートマップ
ax = axes[0]
im = ax.imshow(cm, cmap="Blues")

# 軸ラベル
ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(["Pred 0", "Pred 1"])
ax.set_yticklabels(["True 0", "True 1"])
ax.set_xlabel("Predicted label")
ax.set_ylabel("True label")
ax.set_title("Confusion Matrix (Agent)")

# 値をマスの上に表示
for i in range(2):
    for j in range(2):
        ax.text(
            j, i,
            cm[i, j],
            ha="center", va="center",
            color="black",
            fontsize=11,
        )

fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

# 2) Precision / Recall / F1 / Fbeta / FPR の棒グラフ
ax2 = axes[1]
x = np.arange(len(score_names))
ax2.bar(x, score_values)
ax2.set_xticks(x)
ax2.set_xticklabels(score_names, rotation=30)
ax2.set_ylim(0.0, 1.0)
ax2.set_ylabel("Score")
ax2.set_title("Agent Metrics (eval_df)")

# 値を棒の上に表示（小数3桁）
for i, v in enumerate(score_values):
    ax2.text(i, v + 0.02, f"{v:.3f}", ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()
